### 1. Prompt Chaining:

Prompt Chaining is a techinique in NLP where multiple prompts are sequenced together to guide a model through a complex task or reasoning process. Instead of relying on a single prompt to achieve a desired outcome, prompt chaining breaks the task into smaller , manageable steps, with each step building on the previous one. This approach can improve accuracy, coherence, and control when working with large language models. LangGraph, is a framework designed to facilitate structured interactions with language models, making it an excellent tool for implementing prompt chaining. It allows you to define a graph of nodes (representing individual prompts or tasks) and edges (representing the flow of information between them). This structure enables dynamic, multi-step conversations or workflows, where the output of one node can feed into the input of the next.

### How Prompt Chaining Works with LangGraph

1. *<u>Define the Task</u>*: Start by breaking down the problem into smaller sub-tasks. For example, if you want to generate a detailed report, you might split it into step like 'gather data' , 'analyze data', and 'write summary'.

2. *<u>Create Nodes</u>*: Each sub-task becomes a node in the LangGraph structure. A node could be a prompt that instructs the model to perform a specific action, such as "List key facts about X" or "Summarize the following text".

3. *<u>Establish Edges</u>*: Edges define the sequence and dependencies  between nodes. For instance, the output of the 'gather data' node flows into the 'analyze data' node, ensuring the model has the necessary context to proceed.

4. *<u>Execute the Graph</u>*: LangGraph processes the nodes in order, passing info along the edges. The model generates responses step-by-step, refining the output as it progresses through the chain.

5. *<u>Iterate if Needed</u>*: LangGraph supports conditional logic and loops, so you can revisit earlier nodes or adjust the flow based on intermediate results.

In [8]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model='smollm2:1.7b')
result = llm.invoke("Hi")
result

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'smollm2:1.7b', 'created_at': '2025-09-30T05:11:20.7741553Z', 'done': True, 'done_reason': 'stop', 'total_duration': 6159412700, 'load_duration': 3811496800, 'prompt_eval_count': 30, 'prompt_eval_duration': 1195992200, 'eval_count': 10, 'eval_duration': 1148696700, 'model_name': 'smollm2:1.7b'}, id='run--c82a901b-5bb7-4427-ab5b-5b3129835ec3-0', usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40})

In [ ]:
from typing_extensions import TypedDict

class State(TypedDict):
    topic:str
    story:str
    improved_story:str
    final_story_after_twist:str

def create_story(state:State):
    """Creates story based on the given topic
    args:none
    return: story in the form of string, based on the given topic
    """
    msg = llm.invoke(f"As the japan's best manga and anime story writer expertise in fantasy and adventure genre, write a story premise about {state['topic']}")
    return {'story':msg.content}

def check_conflict(state:State):
    """Checks whether the story premise is according to the given condition or not. If pass, then continue to next node, else create the story premise again.
    args:none
    output: Pass or Fail
    """
    msg = llm.invoke(f"""
                    You are the best manga and anime story premise evaluator who checks for:
                    1. Originality: Does the premise introduce unique elements or concepts that set it apart from existing stories?
                    2. World-building: Does the premise provide enough context and background information about the world in which the
                    story takes place?
                     
                    Evaluate the given below story premise given and **reply only with pass** if it satisfies the above details else **reply me
                    with fail**
                     
                    premise: {state['story']}
                     """)
    if msg.content.lower() == 'pass':
        return "Pass"
    else:
        return 'Fail'
    

def improved(state:State):
    """Return the improved version of the state based on the criteria"""
    msg = llm.invoke(f"""
    Based on the story premise, create me the first part of the story of manga / anime within 500 words.
    story premise: {state['story']}
    """)
    return {'improved_story':msg.content}

def final_story(state:State):
    """Adds some twist to the improved story and returns it"""
    msg = llm.invoke(f"""
                    Now add the below twist to the given improved story:
                     
                    twist: add some twist in the story which is emotionally engaging, with clear emotional arc or character development
                    improved_story: {state['improved_story']}
    """)

    return {'final_story_after_twist':msg.content}



In [13]:
from langgraph.graph import StateGraph, START,END

builder = StateGraph(State)

builder.add_node("story",create_story)
builder.add_node("improved", improved)
builder.add_node("final",final_story)

builder.add_edge(START,"story")
builder.add_conditional_edges("story",check_conflict,{'Pass':'improved','Fail':'story'})
builder.add_edge("improved",'final')
builder.add_edge('final',END)

builder_graph = builder.compile()

In [14]:
from IPython.display import Markdown

response = builder_graph.invoke({'topic':"After getting Banished from the country, I Became overpowered with my Appraisal skill"})
Markdown("# Your Story: ")
Markdown(response['improved_story'])
Markdown(response['final_story_after_twist'])

In the small town of Kokuryu nestled within a vast and ancient forest, there lived a young man named Kaito Yamato who possessed an extraordinary gift - the ability to Appraise any object with complete accuracy and insight. Born into a family of royal lineage, Kaito was once the most powerful ruler in the kingdom, admired by all for his wisdom and leadership. However, fate had other plans as he fell ill and made a crucial decision that led him down a path of exile from his homeland.

Kaito's life took an abrupt turn after he lost his throne. With no use left for his royal title, he was forced to live on the outskirts of Kokuryu, where he struggled to find purpose without the weight of the crown. But what Kaito didn't realize was that this exile had granted him a special gift - the ability to Appraise any object with complete accuracy and insight.

One fateful day, while wandering through the bustling city streets, Kaito stumbled upon an old, worn-out scroll tucked away in a small shop. Intrigued by its age and mysterious aura, he decided to purchase it despite having no use for such an item. Upon opening the scroll, a burst of information filled his mind - it was a cursed magical artifact once wielded by the ancient Kokuryu's king!

Overwhelmed with awe at this revelation, Kaito chose to hide his newfound knowledge and the scroll in a secret location within Kokuryu. As time passed, he found himself increasingly fascinated by the scroll's information - it detailed every object's strength, weaknesses, history, and even its potential for growth. His constant curiosity about this magical artifact led him to master his Appraisal skill to new levels of efficiency.

With each passing day, Kaito became more skilled at understanding the intricacies of objects around him. He began to use his skills not only to survive in his exile but also to help those less fortunate and bring positive change back to Kokuryu. A young girl living in a rural village was struggling to make ends meet as she had no marketable skills or resources. Kaito took it upon himself to mentor her, teaching her how to enhance the natural abilities within her that could be sold for profit.

Another day found him in the city with news of an imminent attack by ruthless bandits who threatened the peaceful life of Kokuryu's citizens. Using his newfound knowledge and powers, he devised a plan to outsmart them. He utilized his understanding of weapons and armor, as well as his magic on certain objects for enhanced strength or durability.

After defeating the bandits with ease, word spread about the young stranger with extraordinary powers who saved Kokuryu. This led to the emergence of new allies and enemies alike - some seeking to protect Kaito's newfound fame, while others aimed to exploit the artifact's power for their own benefit. As a result, more individuals began to approach him with requests ranging from retrieving valuable items lost or stolen to creating customized tools for various tasks.

Kaito navigated through this complex web of intrigue, realizing that his exile was not punishment but an opportunity. His unique position allowed him to serve two purposes: preserving peace in Kokuryu and protecting those around him who were innocent victims of his former status as king. The path ahead would undoubtedly be fraught with challenges and dangers. Yet, armed with the gift of Appraisal and a heart full of empathy for others' plight, Kaito Yamato continued on his journey to make a difference in the world.

But there was one more twist - a sudden turn of events that would change everything: the day an old friend returned from exile, unaware of what he had found in the scroll and its immense power it held. Kaito had grown attached to this individual who brought light into his life during his darkest days as king. This revelation made him question whether he should help his former friend or keep the secret hidden forever.

### Benefits of Prompt Chaining with LangGraph:

* *<u>Improved Context Management</u>*: By breaking tasks into smaller prompts, the model can focus on one aspect at a time, reducing the risk of losing context in long inputs.

* *<u>Modularity</u>*: You can reuse or rearrange nodes for different tasks, making the system flexible,

* *<u>Debugging</u>*: If something goes wrong, it's easier to pinpoint which step failed and adjust the prompt or logic accordingly.

* *<u>Complex Reasoning</u>*: Chaining prompts allows the model to 'think' step-by-step , mimicking human problem-solving more effectively.